## 卷积神经网络

* 1 卷积神经网络学到的模式具有平移不变性（translation invariant）。

* 2 卷积神经网络可以学到模式的空间层次结构<br />
>空间上相邻的像素具有相似的值，RGB的各个通道之间分别有密切的关联性、相距较远的像素之间没有什么关联等。3维形状中可有隐藏有指的提取的本质模式。

> 特征图：对于包含两个空间轴（高度和宽度）和一个深度轴（也叫通道轴）的3D张量，其卷积也叫特征图（feature map），RGB的深度轴=3，黑白图像=1。

> 输出特征图：特征图经过卷积运算之后，所有图块应用相同的变换，生成输出特征图，该输出特征图仍是一个3D张量，具有宽度和高度，其深度可任意取值，因为输出深度是层的参数。深度轴的不同通道不在像RGB输入那样代表特定颜色，而是代表过滤器。过滤器对输入数据的某一方面进行编码，比如，单个过滤器可以从更高层次编码这样一个概念：“输入中包含一张脸”

Q1：相比全连接层，卷积运算的优势在哪？  
A1：全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一纬度的神经元）处理，所以无法利用与形状有关的信息。而卷积层可以保持形状不变。当输入数据是图像时，卷积层会以三维数据的形式接收输入数据，并同样以三维数据的形式输出至下一层。因此在CNN中，可以（有可能）正确理解图像等具有形状的数据。

Q2：卷积与点积之间的差别？  
A2：卷积是相同位置对应的元素乘积之后再求和，点积是矩阵与矩阵的乘法

Q3：输入特征图的深度轴（通道）方向怎么计算？  
A3： 类似2维卷积运算，使用每一个通道的滤波器进行卷积特征图上对应的图块，最终求和输出

Q4：池化处理本质是降维处理

Q5：池化层的特征有哪些？  
A5：（1）没有要学习的参数；（2）通道数不发生变化；（3）对微小的位置变化具有鲁棒性

## Python实现卷积

输入特征图：  
> x = np.random.rand(10,1,28,28)---产生10个高为28、长为28、通道数为1的数据

In [2]:
import numpy as np

In [3]:
x = np.random.rand(10,1,28,28)

In [4]:
x.shape

(10, 1, 28, 28)

要是访问第一个数据的第1个通道的空间数据，可以写成：

In [5]:
x[0,0]
x[0][0]

array([[2.21503744e-01, 3.66624802e-01, 4.11093932e-01, 6.06922952e-01,
        4.23270252e-02, 3.05241915e-01, 4.67229623e-01, 9.32111523e-01,
        9.12955504e-01, 2.95763701e-01, 2.88074756e-01, 5.18029433e-01,
        2.39207456e-01, 7.63350586e-01, 4.87660078e-01, 9.94554107e-01,
        2.86976735e-01, 3.94815334e-01, 5.92998204e-01, 1.08074958e-01,
        5.29543602e-01, 6.87897975e-01, 4.04773825e-01, 3.67285302e-01,
        3.47254759e-01, 8.79817574e-01, 8.69928287e-01, 3.11680476e-01],
       [3.93524107e-01, 7.60465265e-01, 2.58509073e-01, 3.39386720e-02,
        6.66752914e-01, 7.96191480e-01, 9.02720505e-01, 9.98647786e-01,
        7.13395611e-02, 1.09759114e-01, 5.28057440e-01, 2.63943467e-01,
        9.16539367e-01, 8.84508846e-01, 6.18547437e-01, 1.43045113e-01,
        6.64480098e-01, 7.40798009e-01, 6.18257517e-01, 6.75431583e-01,
        4.45915764e-01, 2.58532743e-01, 4.02629236e-01, 9.15861021e-01,
        7.74024828e-01, 6.80010340e-01, 3.89922992e-01, 5.56879

 im2col函数将应用滤波器的区域（3维方块）横向展开为1列,横向怎么理解？  
> 这是为了使用上矩阵的点积运算，输入特征图进行image to column行展开，滤波器进行image to column列展开，然后就可以使用点积进行卷积的算法。

In [6]:
s = np.array([[[[1,2,3],[4,5,6],[7,8,9]]]])

In [7]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充

    Returns
    -------
    col : 2维数组
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [8]:
im2col(input_data=s,filter_h=1,filter_w=1)

array([[1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]])

In [9]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(input_data=x1,filter_h=5, filter_w=5)
print(col1.shape)

(9, 75)


In [10]:
x1 = np.random.rand(10,3,7,7)
col1 = im2col(input_data=x1,filter_h=5, filter_w=5)
print(col1.shape)

(90, 75)


In [11]:
# 实现卷积层

In [12]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    col :
    input_shape : 输入数据的形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad

    Returns
    -------

    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [13]:
    
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中间数据（backward时使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 权重和偏置参数的梯度
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx


In [14]:
# 池化层的实现

In [15]:
import numpy as np

In [16]:
class Pooling:
    
    def __init__(self, pool_h, pool_w, stride=1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    def forward(self,x):
        N,C,H,W = x.shape
        out_h = int(1 +(H-self.pool_h)/self.stride)
        out_w = int(1 +(W - self.pool_w)/self.stride)
        # 展开（1）
        col = im2col(x, sel.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        # 最大值（2）
        col=np.max(col, axis=1)
        # 转换形状
        out = col.reshape(N, out_h, out_w, C).tranpose(0,3,1,2)
        return out

In [17]:
# simpleConvNet

In [18]:
class SimpleConvNet:
    """简单的ConvNet

    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 输入大小（MNIST的情况下为784）
    hidden_size_list : 隐藏层的神经元数量的列表（e.g. [100, 100, 100]）
    output_size : 输出大小（MNIST的情况下为10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 指定权重的标准差（e.g. 0.01）
        指定'relu'或'he'的情况下设定“He的初始值”
        指定'sigmoid'或'xavier'的情况下设定“Xavier的初始值”
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """求损失函数
        参数x是输入数据、t是教师标签
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """求梯度（数值微分）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """求梯度（误差反向传播法）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [19]:
from keras import regularizers

Using TensorFlow backend.


In [20]:
f = np.random.randn(2,2)

In [21]:
f

array([[ 1.08450782, -0.90835725],
       [-0.61847157, -0.78347028]])

In [22]:
from keras import layers
from keras import models

In [27]:
model = models.Sequential()

In [28]:
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(28, 28,1)))

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.add(layers.MaxPooling2D((2,2)))

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.add(layers.MaxPooling2D((2,2)))

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [38]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [39]:
from keras import layers, models

In [40]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28, 1)))

In [41]:
model.add(layers.MaxPooling2D((2, 2)))

In [42]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [43]:
model.add(layers.MaxPool2D((2, 2)))

In [44]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [45]:
# 将3D输出展平为1D，然后再上面添加几个Dense层

In [46]:
model.add(layers.Flatten())

In [47]:
model.add(layers.Dense(64, activation='relu'))

In [48]:
model.add(layers.Dense(10, activation='softmax'))

In [49]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [50]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [51]:
train_images = train_images.reshape((60000, 28, 28,1))

In [52]:
train_images.shape

(60000, 28, 28, 1)

In [53]:
train_images = train_images.astype('float32') / 255

In [54]:
train_images[0][6:7]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.11764706],
        [0.14117648],
        [0.36862746],
        [0.6039216 ],
        [0.6666667 ],
        [0.99215686],
        [0.99215686],
        [0.99215686],
        [0.99215686],
        [0.99215686],
        [0.88235295],
        [0.6745098 ],
        [0.99215686],
        [0.9490196 ],
        [0.7647059 ],
        [0.2509804 ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [55]:
train_labels = to_categorical(train_labels)

由[0-9]的list改成one-hot形式

In [56]:
train_labels[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [57]:
a = [14,5,6,7,8,9,0]

In [58]:
to_categorical(a).shape

(7, 15)

In [59]:
test_labels = to_categorical(test_labels)

In [60]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)


Epoch 1/5
60000/60000 [==============================] - 41s 689us/step - loss: 0.1818 - accuracy: 0.9430
Epoch 2/5
60000/60000 [==============================] - 46s 774us/step - loss: 0.0461 - accuracy: 0.9854
Epoch 3/5
60000/60000 [==============================] - 43s 722us/step - loss: 0.0324 - accuracy: 0.9898
Epoch 4/5
60000/60000 [==============================] - 46s 763us/step - loss: 0.0243 - accuracy: 0.9927
Epoch 5/5
60000/60000 [==============================] - 46s 760us/step - loss: 0.0191 - accuracy: 0.9942s - loss: 0.0191 - accu


#### 验证训练之后的准确度

In [61]:
test_images =test_images.reshape((10000, 28, 28, 1))

In [62]:
test_images = test_images.astype('float32') / 255

In [63]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 344us/step


In [64]:
test_acc

0.9918000102043152

#### 结论：CNN较2层的全连接更容易实现照片信息的处理

### 也可以进行结果的展示

In [70]:
from PIL import Image

In [107]:
test_images.dtype = "float32"

In [108]:
images = test_images

In [109]:
for i in images:
    print(i.shape)
    break

(28, 28, 1)


In [119]:
images.dtype = "float32"

In [120]:
images.dtype

dtype('float32')

In [122]:
for i in range(images.shape[0]):
    if i % 1000 == 0:
        Image.fromarray(images[i].reshape((28,28)), mode="L").save("./result"+str(i)+".png")

In [126]:
%matplotlib notebook